In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import ast
import matplotlib.pyplot as plt
# from together import Together
from alive_progress import alive_bar
from collections import Counter

from prompt import *
from utils import extract_option

PATH = "/home/cc/PHD/HealthBranches/"

In [ ]:
qtc = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_check.csv")
qc = pd.read_csv("/home/cc/PHD/HealthBranches/questions_checked.csv")

In [ ]:
def _evaluate_answers(file_path: str, model: str):
    df = pd.read_csv(file_path)
    accs = []
    
    for col in [c for c in df.columns if c.startswith(("zero_shot", "one_shot"))]:
        df[f'{col}_choice'] = df[col].apply(extract_option)
        df[f'{col}_is_correct'] = df[f'{col}_choice'] == df['real']
        accuracy = df[f'{col}_is_correct'].mean()
        accs.append(accuracy)
    
    accs.insert(0, model)
    return accs

In [ ]:
bench_path = "/home/cc/PHD/HealthBranches/results_misc/results_QUIZ_bench_"
base_path = "/home/cc/PHD/HealthBranches/results_misc/results_QUIZ_baseline2_"
models = ['gemma2_9b']

bench = [_evaluate_answers(f"{bench_path}{model}.csv", model) for model in models]
baseline = [_evaluate_answers(f"{base_path}{model}.csv", model) for model in models]

In [ ]:
baseline

In [ ]:
[['gemma2_9b', np.float64(0.685), np.float64(0.707)]]
[['gemma2_9b', np.float64(0.928), np.float64(0.699), np.float64(0.875)]] #new path
[['gemma2_9b', np.float64(0.914), np.float64(0.695), np.float64(0.877)]] #old path

In [ ]:
### PLOT PRIMA E DOPO DOMANDE DIFF ###

bench_old = [['mistral:7b',
  np.float64(0.1970074812967581),
  np.float64(0.23524522028262676)],
 ['gemma:7b', np.float64(0.24522028262676643), np.float64(0.2743142144638404)],
 ['gemma2:9b',
  np.float64(0.15793848711554448),
  np.float64(0.26101413133832085)],
 ['gemma3:4b',
  np.float64(0.2194513715710723),
  np.float64(0.25353283458021614)],
 ['llama3.1:8b',
  np.float64(0.1770573566084788),
  np.float64(0.21529509559434745)],
 ['qwen2.5:7b',
  np.float64(0.1886949293433084),
  np.float64(0.2502078137988362)],
 ['llama2:7b',
  np.float64(0.18952618453865336),
  np.float64(0.20199501246882792)]]

bench_new = [['mistral:7b',
  np.float64(0.3475238922675934),
  np.float64(0.42224152910512597)],
 ['gemma:7b',
  np.float64(0.38835794960903564),
  np.float64(0.41876629018245004)],
 ['gemma2:9b', np.float64(0.4291920069504778), np.float64(0.4630755864465682)],
 ['gemma3:4b', np.float64(0.4144222415291051), np.float64(0.4283231972198089)],
 ['llama3.1:8b',
  np.float64(0.40834057341442226),
  np.float64(0.45091225021720244)],
 ['qwen2.5:7b',
  np.float64(0.45351867940920937),
  np.float64(0.48479582971329277)],
 ['llama2:7b',
  np.float64(0.17115551694178974),
  np.float64(0.26151172893136404)]]

In [ ]:
def set_neurips_style():
    """
    Apply NeurIPS-friendly styling to matplotlib plots with seaborn darkgrid aesthetics
    and Computer Modern fonts
    """
    # Add Computer Modern font family
    # This assumes the user has the Computer Modern fonts installed in the system
    # If not, you might need to install them or use matplotlib's builtin 'cm' family
    try:
        # Try to use the proper Computer Modern fonts if available
        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['font.serif'] = ['Computer Modern Roman'] + plt.rcParams['font.serif']
        plt.rcParams['mathtext.fontset'] = 'cm'  # Use Computer Modern math font
    except:
        # Fallback to matplotlib's built-in Computer Modern
        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['mathtext.fontset'] = 'cm'
    
    # Figure size for NeurIPS papers (designed to fit in a single column)
    plt.rcParams['figure.figsize'] = (3.5, 2.625)  # 3.5 x 2.625 inches is good for a single column
    
    # Font sizes
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.titlesize'] = 12
    plt.rcParams['axes.labelsize'] = 12
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    plt.rcParams['legend.fontsize'] = 12
    
    # Line widths
    plt.rcParams['axes.linewidth'] = 0.8
    plt.rcParams['lines.linewidth'] = 1.5
    plt.rcParams['grid.linewidth'] = 0.5
    
    # Marker size and style
    plt.rcParams['lines.markersize'] = 4
    plt.rcParams['scatter.marker'] = 'o'
    
    # Seaborn-inspired colors - maintains colorblind friendliness
    plt.rcParams['axes.prop_cycle'] = plt.cycler('color', [
        '#4C72B0',  # blue
        '#DD8452',  # orange
        '#55A868',  # green
        '#C44E52',  # red
        '#8172B3',  # purple
        '#937860',  # brown
        '#DA8BC3',  # pink
        '#8C8C8C',  # gray
        '#CCB974',  # khaki
        '#64B5CD',  # light blue
    ])
    
    # Darkgrid background and grid (seaborn style)
    plt.rcParams['axes.facecolor'] = '#EAEAF2'  # light gray background
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['grid.color'] = 'white'
    plt.rcParams['grid.alpha'] = 1.0
    plt.rcParams['grid.linewidth'] = 1.0
    
    # Spines and ticks - seaborn darkgrid typically has reduced spines
    plt.rcParams['axes.spines.top'] = False
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.left'] = True
    plt.rcParams['axes.spines.bottom'] = True
    # plt.rcParams['axes.spines.color'] = ['#CCCCCC']  # Light gray spines
    
    plt.rcParams['xtick.direction'] = 'out'
    plt.rcParams['ytick.direction'] = 'out'
    plt.rcParams['xtick.major.width'] = 0.8
    plt.rcParams['ytick.major.width'] = 0.8
    plt.rcParams['xtick.color'] = '#555555'
    plt.rcParams['ytick.color'] = '#555555'
    
    # Legend
    plt.rcParams['legend.frameon'] = True
    plt.rcParams['legend.framealpha'] = 0.9
    plt.rcParams['legend.edgecolor'] = '#CCCCCC'
    
    # LaTeX-like rendering for text with Computer Modern font
    plt.rcParams['text.usetex'] = True  # Set to True if you have LaTeX installed
    
    # Saving options
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['savefig.bbox'] = 'tight'
    plt.rcParams['savefig.pad_inches'] = 0.05

    # set figure dpi
    plt.rcParams['figure.dpi'] = 300

def use_latex_with_computer_modern():
    """
    Alternative function to use actual LaTeX for text rendering.
    This requires a working LaTeX installation with the Computer Modern fonts.
    """
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
    })
    
    # Call the rest of the styles
    set_neurips_style()

In [ ]:
def plot_by_models(bench_old: list[list], bench_new: list[list]):
    use_latex_with_computer_modern()

    # Sort both datasets by the first metric (e.g., Zero Shot) ascending
    bench_old_sorted = sorted(bench_old, key=lambda x: x[1])
    bench_new_sorted = sorted(bench_new, key=lambda x: x[1])

    # Extract labels and values for old data
    labels = [entry[0] for entry in bench_old_sorted]
    old_vs1 = [entry[1]*100 for entry in bench_old_sorted]
    old_vs2 = [entry[2]*100 for entry in bench_old_sorted]

    # Map new data to the old labels order
    new_map = {entry[0]: entry for entry in bench_new_sorted}
    new_vs1 = [new_map[name][1]*100 for name in labels]
    new_vs2 = [new_map[name][2]*100 for name in labels]

    # Bar settings
    width = 0.15  # narrower bars to fit five series
    num_series = 4

    # Spacing between groups
    group_spacing = width * (num_series+0.75)
    x = np.arange(len(labels)) * group_spacing

    # Create figure and axes
    fig, ax = plt.subplots(figsize=(12, 7))

    # Plot grouped bars: old/new × two metrics
    bars_old_vs1 = ax.bar(x - 1.5*width, old_vs1, width, label="Old Zero Shot")
    bars_new_vs1 = ax.bar(x - 0.5*width, new_vs1, width, label="New Zero Shot")
    bars_old_vs2 = ax.bar(x + 0.5*width, old_vs2, width, label="Old Zero Shot + RAG")
    bars_new_vs2 = ax.bar(x + 1.5*width, new_vs2, width, label="New Zero Shot + RAG")

    # Configure ticks and labels
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation="vertical", ha="center")
    ax.set_xlabel("Models")
    ax.set_ylabel("Accuracy")

    # Place legend inside plot at bottom right
    ax.legend(loc='lower right')

    # Annotate bars with values (no leading zero)
    for bars, vals in zip(
        (bars_old_vs1, bars_new_vs1, bars_old_vs2, bars_new_vs2),
        (old_vs1, new_vs1, old_vs2, new_vs2)
    ):
        labels_txt = [f"{v:.0f}" for v in vals]
        ax.bar_label(bars, labels=labels_txt, padding=2, fontsize=8)

    plt.tight_layout()
    plt.savefig("plot_models_diff.png", dpi=500, bbox_inches='tight')
    plt.show()

In [ ]:
plot_by_models(bench_old, bench_new)

### PLOT PRIMA E DOPO DOMANDE DIFF (FINE) ###

In [ ]:
results_new = pd.read_csv("/home/cc/PHD/HealthBranches/results.csv")

In [ ]:
path1 = pd.read_csv("/home/cc/PHD/HealthBranches/results_misc/results_QUIZ_baseline_gemma2_9b.csv")
path2 = pd.read_csv("/home/cc/PHD/HealthBranches/results_misc/results_QUIZ_bench_gemma2_9b.csv")

In [ ]:
sam = np.random.randint(0, 1000)

print(path1.loc[sam]['path'])
print(path2.loc[sam]['path'])


In [ ]:
ds = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_qwen2.5_72b.csv")
cnt = 0

for index, row in ds.iterrows():
    if extract_option(row['zero_shot']) not in ['A', 'B', 'C', 'D', 'E']:
        cnt +=1
        print(row['zero_shot'])
print(cnt)

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_qwen2.5_7b.csv")

In [ ]:
# 2. Compute path length as the number of '->' in each string
df['path_len'] = df['path'].str.count(r'->')

# 3. Group by that new column
grouped = df.groupby('path_len')

# 4a. If you just want to see group sizes:
print("Rows per path-length:")
print(grouped.size().sort_index())

# 4b. If you want to iterate through each group:
for length, group in grouped:
    print(f"\n=== Path length = {length} (i.e. {length} '->' separators) ===")
    print(group[['path']])

In [ ]:
# 2. Compute path length (number of '->')
df['path_len'] = df['path'].str.count(r'->')

# 3. Apply extract_option to 'answer' and compare to 'real'
#    (Assuming extract_option is already defined or imported)
df['pred'] = df['zero_shot'].apply(extract_option)
df['correct'] = df['pred'] == df['real']

# 4. Group by path length and count matches & totals
agg = (
    df
    .groupby('path_len')
    .agg(
        total_rows=('correct','size'),
        correct_count=('correct','sum')
    )
    .sort_index()
    .reset_index()
)

total_rows_all = len(df)
agg['accuracy'] = agg['correct_count'] / total_rows_all

# 7) Plot bar chart of weighted accuracies
labels = [str(length) for length in agg.index]
weighted_accuracies = agg['accuracy']

plt.figure(figsize=(12, 8))
bars = plt.bar(labels, weighted_accuracies)
plt.xlabel('Path Length (number of "->" separators)')
plt.ylabel('Weighted Accuracy\n(corrects ÷ total rows)')
plt.title('Weighted Accuracy by Path Length')
plt.ylim(0, 1.0)
plt.xticks(rotation=45)
plt.tight_layout()

# Annotate bars with percentages
for bar, val in zip(bars, weighted_accuracies):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        val + 0.005,
        f"{val:.0%}",
        ha='center',
        va='bottom'
    )

plt.show()

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/final_dataset.csv")

In [ ]:
# 1) Carica il CSV con le domande da escludere
df_blocklist = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_remove.csv")

# 2) Carica il CSV originale che vuoi filtrare
df_orig = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv")

# 3) Filtra: tieni solo le righe il cui 'question' NON è nella blocklist
df_filtrato = df_orig[~df_orig['question'].isin(df_blocklist['question'])]

# 4) Scrivi il risultato su un nuovo CSV
df_filtrato.to_csv('tutte_le_domande_filtrate.csv', index=False)
print(len(df_filtrato))

In [ ]:
def is_valid_options(opt_str):
    """
    Ritorna True se opt_str può essere convertito in lista di lunghezza 5,
    altrimenti False (anche in caso di eccezione di parsing).
    """
    try:
        # Normalizza le virgolette e tenta l'ast.literal_eval
        opts = ast.literal_eval(
            opt_str
            .replace("['", '["')
            .replace("']", '"]')
            .replace("', '", '", "')
        )
        # Controlla che sia lista di lunghezza 5
        return isinstance(opts, list) and len(opts) == 5
    except (ValueError, SyntaxError):
        return False

# Applichiamo il filtro sul DataFrame originale
mask = df['options'].apply(is_valid_options)

# df filtrato: mantengo solo le righe valide
df = df.loc[mask].reset_index(drop=True)

In [ ]:
df.to_csv("/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv", index=False)

In [ ]:
i = 0
for c in df['correct_option'].values:
    if c not in ["A", "B", "C", "D", "E"]:
        print(i)
        print(df.loc[i]['correct_option'])
    i = i+1

In [ ]:
df = pd.read_csv(f"{PATH}results/results_quiz_Llama-3.3-70B-Instruct-Turbo-Free.csv")
df["zero_shot_new"] = df["zero_shot"].apply(extract_option)

big_filtered = df[df["zero_shot_new"].str.upper() != df["real"].str.upper()]
len(big_filtered)

In [ ]:
small_df = pd.read_csv(f"{PATH}results/results_quiz_mistral.csv")
small_df = small_df[small_df["question"].isin(big_filtered["question"])]

In [ ]:
small_df["zero_shot_new"] = small_df["zero_shot"].apply(extract_option)
small_filtered = small_df[small_df["zero_shot_new"].str.upper() != small_df["real"].str.upper()]

In [ ]:
# Directory contenente i file CSV
csv_directory = "/home/cc/PHD/HealthBranches/quiz-ultimate-dataset"

# Leggere tutti i CSV
csv_files = [os.path.join(csv_directory, f) for f in os.listdir(csv_directory) if f.endswith(".csv")]
questions = []

print(csv_files)

# Elaborazione di ogni CSV
for file in csv_files:
    df = pd.read_csv(file)

    # Rimuove la colonna "zero_shot_rag" se esiste
    if "zero_shot_rag" in df.columns:
        df = df.drop(columns=["zero_shot_rag"])

    # Applica la funzione di estrazione su zero_shot
    df["zero_shot"] = df["zero_shot"].apply(extract_option)

    # Filtra le righe in cui zero_shot è diverso da real
    df_filtered = df[df["zero_shot"] != df["real"]].copy()

    questions.extend(df_filtered['question'].values)

In [ ]:
# Conta le occorrenze
conteggio = Counter(questions)

# Ordina in ordine decrescente
dizionario_ordinato = dict(sorted(conteggio.items(), key=lambda x: x[1], reverse=True))

filtered_dict = {k: v for k, v in dizionario_ordinato.items() if v >= 6}

print(len(filtered_dict))

In [ ]:
questions_to_check = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_check.csv")

# Trova le domande che sono sia nel dizionario che nel DataFrame
matching_questions = [q for q in filtered_dict.keys() if q in questions_to_check["question"].values]
print(len(matching_questions))

print("Domande trovate nel DataFrame:", len(matching_questions))

In [ ]:
# Carica il CSV (sostituisci 'file.csv' con il tuo file)
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_checker_70B.csv")

# Conta le occorrenze di ogni valore nella colonna "check"
conteggio = df["check"].value_counts().sort_index()

# Stampa il risultato
print(conteggio)

In [ ]:
import pandas as pd
import ast
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# Se la GPU è disponibile, usa 'cuda', altrimenti 'cpu'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Esecuzione su: {device}")

# Carica il modello sulla GPU se disponibile
model = SentenceTransformer('all-mpnet-base-v2', device=device)

# Sostituisci 'data.csv' con il percorso del tuo file CSV.
df = pd.read_csv('/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv')

# Definisce la soglia di similarità (80%)
SIMILARITY_THRESHOLD = 0.9

# Pre-elabora il DataFrame:
# 1. Dividi il campo 'path' in una lista di segmenti
df['path_segments'] = df['path'].apply(lambda x: [seg.strip() for seg in x.split("->")])
# 2. Estrai il testo dell'opzione corretta
def extract_correct_option(row):
    try:
        options_list = ast.literal_eval(row['options'].replace("['", '["').replace("']", '"]').replace("', '", '", "'))
    except (ValueError, SyntaxError) as e:
        # Se il parsing fallisce, restituisce una stringa vuota
        return ""
    correct_letter = row['correct_option'].strip().upper()
    index = ord(correct_letter) - ord('A')
    if index < 0 or index >= len(options_list):
        return ""
    return options_list[index]

df['correct_option_text'] = df.apply(extract_correct_option, axis=1)

# Elabora in batch gli embedding delle opzioni corrette
correct_texts = df['correct_option_text'].tolist()
correct_embeddings = model.encode(correct_texts, device=device, convert_to_tensor=True)

# Funzione per verificare in una riga se almeno un segmento del path è semanticamente simile all'opzione corretta
def check_similar_option_batch(idx, row):
    segments = row['path_segments']
    # Se non ci sono segmenti, ritorna False
    if not segments:
        return False
    # Calcola in batch gli embedding dei segmenti (convert_to_tensor=True per operare su GPU)
    segments_embeddings = model.encode(segments, device=device, convert_to_tensor=True)
    # Recupera l'embedding dell'opzione corretta per la riga corrente
    correct_emb = correct_embeddings[idx]
    # Normalizza gli embedding per applicare correttamente il calcolo della similarità coseno
    correct_norm = correct_emb / correct_emb.norm()
    segments_norm = segments_embeddings / segments_embeddings.norm(dim=1, keepdim=True)
    # Calcola la similarità coseno tra l'opzione e ogni segmento
    similarities = torch.matmul(segments_norm, correct_norm)
    # Se almeno una similarità supera la soglia, ritorna True
    return (similarities >= SIMILARITY_THRESHOLD).any().item()

# Processa ogni riga e determina se il match semantico supera la soglia
results = [check_similar_option_batch(idx, row) for idx, row in tqdm(df.iterrows())]
df['is_similar_in_path'] = results

# Calcola e stampa il contatore finale
counter = sum(results)
print(f"Numero di righe in cui almeno un segmento del path risulta simile (>= 80%) all'opzione corretta: {counter}")

# Visualizza il DataFrame aggiornato
print(df)